# Nipams Perform Experiments & Collect Results

In [ ]:
RUN_NAME = "all200_feb24"
RUN_NAME_PREV = "all200_dec13"
# MAT_PATTERN = '(LV|res|rec|bre)\d+.mat'
MAT_PATTERN = '\d+.mat'
LIMIT = 200
REPROCESS = False
# FILE_PROCESSED = f'data/interim/{RUN_NAME}.pickle'
FILE_PROCESSED = f'data/interim/{RUN_NAME_PREV}.pickle'
GBQ_RESULTS_TABLE = f'predictions.results_{RUN_NAME}'

# 0. Initialize

In [ ]:
#@title Connect to gDrive & Load Files
ENV_COLAB = 'google.colab' in str(get_ipython())
if ENV_COLAB:
    print('Running on CoLab')
    PROJECT_DRIVE_FOLDER= "/content/drive/MyDrive/NiPAMS/BP development/analysis/nipams-bp-py-main/" #@param {"type":"string"}
    ## Load this for local development version of library.. Loads from GoogleDrive (changes persist)
    from google.colab import drive, widgets, data_table; 
    drive.mount('/content/drive');
    %cd "$PROJECT_DRIVE_FOLDER"
  
    !pip install -qr requirements.txt
    !pip install -q envyaml
    import os, sys; 
    from envyaml import EnvYAML; env = EnvYAML('env/default.yml'); 
    for k,v in env.export().items(): os.environ[k]=str(v)
    sys.path.append(os.path.abspath("./src"))

else:
    print('Not running on CoLab')
    %cd /app
!pwd

from importlib import reload;
from utils.constants import *

Running on CoLab
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1MnayBGJfAervEVUyYNEibuPedW7NJuzm/NiPAMS/BP development/analysis/nipams-bp-py-main
/content/drive/.shortcut-targets-by-id/1MnayBGJfAervEVUyYNEibuPedW7NJuzm/NiPAMS/BP development/analysis/nipams-bp-py-main


# 1. Load & Process Data

In [ ]:
from data import load_data; reload(load_data)
from data import preprocess;

if REPROCESS:
  _dfBp, _dfImu = load_data.load_dataframe_from_mat(folder='data/raw_mat/', pattern=MAT_PATTERN, limit_files=LIMIT)

  dfAll = preprocess.merge_imu_vcg_with_heartbeats(_dfBp, _dfImu)
  dfAll[IMU_DATA_COLS] = dfAll[IMU_DATA_COLS].astype(np.float32)
  dfAll[BP_COLS] = dfAll[BP_COLS].astype(np.float32)
  dfAll.to_pickle(path=FILE_PROCESSED)

In [ ]:
# dfBp, dfImu = load_data.load_dataframe_from_mat(folder='data/raw_mat/', pattern='LLV', limit_files=30)
dfAll = load_data.load_dataframe_from_pickle(path=FILE_PROCESSED)
dfImu = dfAll[INDICIES + INFO_COLS + IMU_COLS]
dfBp = dfAll[INDICIES + INFO_COLS + BP_COLS]

display('------ dfAll ------')
display("Unique Files : ", dfAll.file.nunique())
display(np.unique(dfAll.file))
display(dfAll)

'------ dfAll ------'

'Unique Files : '

199

array(['sub019_HLV1.mat', 'sub019_HLV2.mat', 'sub019_HLV3.mat',
       'sub019_LLV1.mat', 'sub019_LLV2.mat', 'sub019_LLV3.mat',
       'sub019_bre1.mat', 'sub019_rec1.mat', 'sub019_rec2.mat',
       'sub019_res1.mat', 'sub019_res2.mat', 'sub112_HLV1.mat',
       'sub112_HLV2.mat', 'sub112_HLV3.mat', 'sub112_LLV1.mat',
       'sub112_LLV2.mat', 'sub112_LLV3.mat', 'sub112_bre1.mat',
       'sub112_rec1.mat', 'sub112_rec2.mat', 'sub112_res1.mat',
       'sub112_res2.mat', 'sub115_HLV1.mat', 'sub115_HLV2.mat',
       'sub115_HLV3.mat', 'sub115_LLV1.mat', 'sub115_LLV2.mat',
       'sub115_LLV3.mat', 'sub115_bre1.mat', 'sub115_rec1.mat',
       'sub115_rec2.mat', 'sub115_res1.mat', 'sub115_res2.mat',
       'sub133_HLV1.mat', 'sub133_HLV2.mat', 'sub133_HLV3.mat',
       'sub133_LLV1.mat', 'sub133_LLV2.mat', 'sub133_LLV3.mat',
       'sub133_bre1.mat', 'sub133_rec1.mat', 'sub133_rec2.mat',
       'sub133_res1.mat', 'sub133_res2.mat', 'sub176_HLV1.mat',
       'sub176_HLV2.mat', 'sub176_HLV3.m

,file,ax,ay,az,gx,gy,gz,heartbeat,patient,test_type,test_num,tsCC,ts,pp,sbp,dbp
0,sub019_HLV1.mat,-1.089362,1.668758,-9.832985,1.037598,0.259399,0.053406,3.0,sub019,HLV,1,True,0 days 00:00:01.991000,62.045975,122.860863,60.814892
1,sub019_HLV1.mat,-1.012747,1.644816,-9.797072,1.289368,0.091553,-0.053406,3.0,sub019,HLV,1,True,0 days 00:00:01.996000,62.045975,122.860863,60.814892
2,sub019_HLV1.mat,-1.012747,1.632845,-9.829394,1.144409,0.480652,-0.221252,3.0,sub019,HLV,1,True,0 days 00:00:02.001000,62.045975,122.860863,60.814892
3,sub019_HLV1.mat,-1.023521,1.663970,-9.848548,1.296997,-0.476837,-0.026703,3.0,sub019,HLV,1,True,0 days 00:00:02.006000,62.045975,122.860863,60.814892
4,sub019_HLV1.mat,-1.039083,1.649605,-9.947906,1.518250,-0.831604,-0.411987,3.0,sub019,HLV,1,True,0 days 00:00:02.011000,62.045975,122.860863,60.814892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5109758,sub267_HLV2.mat,1.173159,1.845929,-9.873687,-0.007629,2.197266,0.091553,25.0,sub267,HLV,2,True,0 days 00:00:57.805000,64.464264,147.161850,82.697586
5109759,sub267_HLV2.mat,1.189918,1.874660,-9.897629,-0.526428,2.258301,0.106812,25.0,sub267,HLV,2,True,0 days 00:00:57.810000,64.464264,147.161850,82.697586
5109760,sub267_HLV2.mat,1.205480,1.856703,-9.850942,-0.488281,2.281189,0.007629,25.0,sub267,HLV,2,True,0 days 00:00:57.815000,64.464264,147.161850,82.697586
5109761,sub267_HLV2.mat,1.189918,1.860294,-9.842562,-0.503540,2.326965,0.198364,25.0,sub267,HLV,2,True,0 days 00:00:57.820000,64.464264,147.161850,82.697586


In [ ]:
import plotly.express as px
fig = px.line(
    dfAll.head(10000).select_dtypes(include=np.number).sort_values('ts').drop('ts',axis=1),
    template = "plotly_white",
)
# fig.update_layout({
#   'plot_bgcolor': 'rgba(255, 255, 255, 1)',
#   'paper_bgcolor': 'rgba(255, 255, 255, 1)',
#   'paper_bgcolor': 'rgba(255, 255, 255, 1)',
# })
fig

Output hidden; open in https://colab.research.google.com to view.

# 2. Experiments

In [ ]:
import json
from models import experiments; reload(experiments)

<module 'models.experiments' from '/content/drive/.shortcut-targets-by-id/1MnayBGJfAervEVUyYNEibuPedW7NJuzm/NiPAMS/BP development/analysis/nipams-bp-py-main/src/models/experiments.py'>

In [ ]:
splits_random = experiments.split_by_random(dfBp=dfBp)
splits_group = experiments.split_by_group(dfBp=dfBp, **{**json.loads(experiments.DEFAULTS['split_by_group'])['kwargs'], 'split_kwargs':{'n_splits':3}})
splits_query = experiments.split_by_query(dfBp=dfBp, **json.loads(experiments.DEFAULTS['split_by_query'])['kwargs'])

In [ ]:
# # splitting_config = json.loads(experiments.DEFAULTS['split_by_random'])
# splitting_config = json.loads(experiments.DEFAULTS['split_by_group'])
# sampleRandTestInds = getattr(experiments, splitting_config['function'])(dfBp=dfBp, **splitting_config['kwargs'])

# sampleRandTestInds = [{
#     'train':x['train'].drop_duplicates(INDICIES),
#     'test':x['test'].drop_duplicates(INDICIES)
# } for x in sampleRandTestInds]
# # sampleRandTestInds[0]['train'].drop_duplicates(INDICIES)

# sampleExpDfs = experiments.get_experiment(sampleRandTestInds[0], dfImu, dfBp)

# 3. Modeling Pipelines (Baselines & Analytical)

In [ ]:

from models import baselines, feature_models; reload(feature_models)
import sklearn
from sklearn import pipeline, feature_selection, linear_model, ensemble
# from sklearn.pipeline import Pipeline

from models import baselines, feature_models; reload(feature_models)
from features import simple as features_simple, tsfel_vectorizer; reload(tsfel_vectorizer);


In [ ]:
#### Baselines
pipe_mean = pipeline.Pipeline([('transform', preprocess.TransformerTimeSeriesTo3D()),('baselines.DummyRegressor', baselines.DummyRegressor())])
pipe_random = pipeline.Pipeline([('transform', preprocess.TransformerTimeSeriesTo3D()),('baselines.RandomRegressor', baselines.RandomRegressor())])


In [ ]:
#### Simple Feature Methods
vectorizer_simple = preprocess.FunctionTransformer(partial(features_simple.vectorize_mean_std))
pipe_vect_simple_linear = pipeline.Pipeline([
    ('features_simple.vectorize_mean_std', vectorizer_simple),
    ('normalize_features',sklearn.preprocessing.StandardScaler()),
    ('model_Linear', sklearn.linear_model.LinearRegression())
])

pipe_vect_simple_rf = pipeline.Pipeline([
    ('features_simple.vectorize_mean_std', vectorizer_simple),
    ('model_RandomForest', sklearn.ensemble.RandomForestRegressor())
])


In [ ]:
%%time
#### Advanced Feature Methods
# REPROCESS = True #@param{"type":"boolean"}
# FEATURE_DOMAINS = ['statistical']
FEATURE_DOMAINS = ['statistical','temporal','spectral']
FILE_FEATURES = f"data/vectors/{RUN_NAME}.feather"

if REPROCESS:
  dfVectors = tsfel_vectorizer.panelTSDataToVectors(dfAll, domains=FEATURE_DOMAINS, fs=1)
  dfVectors.astype(np.float32).reset_index().to_feather(path=FILE_FEATURES)

dfVects = pd.read_feather(FILE_FEATURES)

# dfVects = pd.concat([
#     pd.read_feather(f"data/vectors/dfImuVects-temporal-{PATTERN}.feather").set_index(INDICIES),
#     pd.read_feather(f"data/vectors/dfImuVects-statistical-{PATTERN}.feather").set_index(INDICIES)
# ], axis=1).reset_index()
dfVects.file = dfVects.reset_index().file.str.extract(pat='(sub[0-9]+_\w+.mat)$')
dfVects = dfVects.set_index(INDICIES)
tsVectorLookupTransform = sklearn.preprocessing.FunctionTransformer(tsfel_vectorizer.VectorLookup(dfVects))

pipe_vect_RF = pipeline.Pipeline([
    ('vectorize_lookup', tsVectorLookupTransform), 
    ('feat_select_simple', sklearn.feature_selection.VarianceThreshold()),
    ('feat_select_lin30', sklearn.feature_selection.SelectFromModel(sklearn.linear_model.LinearRegression(), max_features=30)),
    ('normalize_features',sklearn.preprocessing.StandardScaler()),
    # ('feat_select_mutualinfo50', sklearn.feature_selection.SelectKBest(sklearn.feature_selection.mutual_info_regression, k=50)),
    ('model_randomforest', sklearn.ensemble.RandomForestRegressor())
])
pipe_vect_GradBoost = pipeline.Pipeline([
    ('vectorize_lookup', tsVectorLookupTransform), 
    ('feat_select_simple', sklearn.feature_selection.VarianceThreshold()),
    ('feat_select_lin30', sklearn.feature_selection.SelectFromModel(sklearn.linear_model.LinearRegression(), max_features=30)),
    ('normalize_features',sklearn.preprocessing.StandardScaler()),
    ('model_gradientboost', sklearn.ensemble.GradientBoostingRegressor())
])

FileNotFoundError: ignored

In [ ]:
tsExplodeTransformAxAz = preprocess.FunctionTransformer(partial(preprocess.explode_3d, data_cols=['ax', 'az']))
#### Analytical Approaches
from models import analytical_mvd; reload(analytical_mvd)

pipe_mvd = pipeline.Pipeline([('transform_3d_AxAz', tsExplodeTransformAxAz), ('model_analytical_calibrate', analytical_mvd.AnalyticalBPEstimator())])

In [ ]:

#### Deep Learning Approaches
# !pip install tsai
from models import nn_tsai; reload(nn_tsai)
model_tsai = nn_tsai.TSAI_InceptionTime(n_epoch=11)

tsExplodeTransform = preprocess.FunctionTransformer(partial(preprocess.explode_3d, data_cols=IMU_DATA_COLS))
tsReshape = preprocess.FunctionTransformer(lambda x : x.transpose(0,2,1))

pipe_TSAI = pipeline.Pipeline([('transform_3d', tsExplodeTransform),('transform_reshape', tsReshape), ('model_inceptiontime', model_tsai)])

# Run Experiments

## Individual Experiment

In [ ]:
# dfResults = experiments.run_experiment("tmp", dfImu, dfBp, pipe_mvd, splits_random[:1])
# dfResults = experiments.run_experiment("tmp", dfImu, dfBp, pipe_vect_GradBoost, splits_random)
dfResults = experiments.run_experiment("tmp", dfImu, dfBp, pipe_TSAI, splits_random[:1])

display(dfResults)
dfResults.plot.scatter('sbp','sbp_preds', title='Random Forest Model')

## Many Experiments

In [ ]:
# !pip install google-cloud-bigquery
# !pip install -U pandas-gbq
# !pip install pandas-gbq -U

In [ ]:
# arr_breath_types = [HLV,LLV,RES,BRE,REC,MIX]
# arr_data_size = [sm, md, lg, all]
arr_splits = {
  "splits_random": splits_random, 
  "splits_query": splits_query, 
  "splits_group": splits_group,
}
arr_pipelines = {
  # "mean" : pipe_mean,
  # "random" : pipe_random,
  # "analytical" : pipe_mvd,
  # "vect_simple_linear" : pipe_vect_simple_linear,
  # "vect_simple_rf" : pipe_vect_simple_rf,
  # "vect_RF" : pipe_vect_RF,
  # "vect_GradBoost" : pipe_vect_GradBoost,
  "nn_tsai" : pipe_TSAI,
}

bigquery_schema = [
    {'name':'file', 'type' : 'STRING'},
    {'name':'heartbeat', 'type' : 'FLOAT'},
    {'name':'patient', 'type' : 'STRING'},
    {'name':'test_type', 'type' : 'STRING'},
    {'name':'test_num', 'type' : 'STRING'},
    {'name':'pp', 'type' : 'FLOAT'},
    {'name':'sbp', 'type' : 'FLOAT'},
    {'name':'dbp', 'type' : 'FLOAT'},
    {'name':'fold_number', 'type' : 'FLOAT'},
    {'name':'dbp_actuals', 'type' : 'FLOAT'},
    {'name':'dbp_preds', 'type' : 'FLOAT'},
    {'name':'sbp_actuals', 'type' : 'FLOAT'},
    {'name':'sbp_preds', 'type' : 'FLOAT'},
    {'name':'pp_actuals', 'type' : 'FLOAT'},
    {'name':'pp_preds', 'type' : 'FLOAT'},
    {'name':'experiment_id', 'type' : 'INTEGER'},
    {'name':'pipeline', 'type' : 'STRING'},
    {'name':'uploaded_time', 'type' : 'TIMESTAMP'},
]

import datetime
def run_and_save_experiment(id, dfImu, dfBp, pipe, split, dataset='predictions.results'):
  dfResults = experiments.run_experiment(id, dfImu, dfBp, pipe, split)
  dfResults = dfResults.assign(uploaded_time = datetime.datetime.now())
  dfResults = dfResults.fillna(value=np.NaN)

  dfResults.to_csv(f'data/predictions/{id} - date:{datetime.datetime.now()}.csv')
  dfResults.to_gbq(f'predictions.results_{PATTERN}','nipams',
                        if_exists='append', table_schema=None,
                        # if_exists='append', table_schema=bigquery_schema,
                  )


for split_name, split in arr_splits.items():
  for pipe_name, pipe in arr_pipelines.items():
    experiment_name = f'data:{RUN_NAME} - split:{split_name} - pipe:{pipe_name}'
    print("------------------", experiment_name, "------------------")
    print(f"Started : {datetime.datetime.now()}")
    
    dfResults = experiments.run_experiment(experiment_name, dfImu, dfBp, pipe, split)
    dfResults = dfResults.assign(uploaded_time = datetime.datetime.now())
    dfResults = dfResults.fillna(value=np.NaN)
    dfResults.to_csv(f'data/predictions/{experiment_name} - date:{datetime.datetime.now()}.csv')
    dfResults.to_gbq(GBQ_RESULTS_TABLE,'nipams',
                if_exists='append', table_schema=None,
                # if_exists='append', table_schema=bigquery_schema,
    )
    
    # try:
    # run_and_save_experiment(experiment_name, dfImu, dfBp, pipe, split)
    # except Exception as e:
    #   print("ERROR -", e)
    print(f"Ended : {datetime.datetime.now()}")

# experiments.run_and_save_experiment('split:first50%perfile - model:baseline.random', dfImu, dfBp, pipe_random, splits_random)
# experiments.run_and_save_experiment('split:first50%perfile - model:tree', dfImu, dfBp, pipeVectRF, splits_random)
# experiments.run_and_save_experiment('split:first50%perfile - model:tree.gradboost', dfImu, dfBp, pipeVectGradBoost, splits_random)


------------------ data:all200_feb24 - split:splits_random - pipe:nn_tsai ------------------
Started : 2023-02-24 19:29:10.176511
TSAI.fit X object (22642, 7, 190)
TSAI.fit y float32 (22642,)
TSAI.fit X float32 (22642, 6, 190)
TSAI.fit y float32 (22642,)


epoch,train_loss,valid_loss,mae,_rmse,time
0,0.565390,0.636797,0.596661,0.797996,00:21
1,0.422547,0.397064,0.466637,0.630130,00:21
2,0.413748,0.344758,0.447641,0.587161,00:21
3,0.357801,0.331234,0.428000,0.575529,00:21
4,0.292551,0.275514,0.386531,0.524894,00:21
5,0.251527,0.249804,0.367895,0.499804,00:21
6,0.218844,0.208023,0.333631,0.456096,00:21
7,0.193651,0.187610,0.316368,0.433139,00:21
8,0.155782,0.171412,0.300799,0.414020,00:21
9,0.144993,0.161418,0.290293,0.401768,00:21


mean_absolute_error, r2_score :  2.963 0.812
TSAI.fit X object (22642, 7, 190)
TSAI.fit y float32 (22642,)
TSAI.fit X float32 (22642, 6, 190)
TSAI.fit y float32 (22642,)


epoch,train_loss,valid_loss,mae,_rmse,time
0,0.534979,0.428888,0.495972,0.654895,00:21
1,0.432543,0.470907,0.527748,0.686227,00:22
2,0.382531,0.354223,0.449708,0.595166,00:22
3,0.353113,0.284269,0.402597,0.533169,00:22
4,0.292075,0.276209,0.390644,0.525556,00:21
5,0.254697,0.224770,0.347985,0.474099,00:22
6,0.206665,0.197635,0.324869,0.444562,00:22
7,0.164790,0.177471,0.306119,0.421273,00:21
8,0.143193,0.168105,0.300492,0.410006,00:21
9,0.126017,0.163264,0.293914,0.404059,00:21


mean_absolute_error, r2_score :  2.893 0.809
TSAI.fit X object (22643, 7, 190)
TSAI.fit y float32 (22643,)
TSAI.fit X float32 (22643, 6, 190)
TSAI.fit y float32 (22643,)


...

mean_absolute_error, r2_score :  14.065 -1.33
TSAI.fit X object (20293, 7, 190)
TSAI.fit y float32 (20293,)
TSAI.fit X float32 (20293, 6, 190)
TSAI.fit y float32 (20293,)


epoch,train_loss,valid_loss,mae,_rmse,time
0,0.455123,0.413736,0.480786,0.643223,00:19
1,0.377241,0.317087,0.421518,0.563105,00:19
2,0.302384,0.310592,0.427427,0.557308,00:19
3,0.257682,0.267979,0.386216,0.517667,00:19
4,0.243326,0.215386,0.334726,0.464097,00:19
5,0.225089,0.187189,0.309744,0.432653,00:19
6,0.179061,0.200755,0.320126,0.448056,00:19
7,0.170319,0.156319,0.283385,0.395372,00:19
8,0.128693,0.139516,0.266276,0.373519,00:19
9,0.118012,0.131533,0.259889,0.362675,00:19


mean_absolute_error, r2_score :  15.603 -4.378


100%|██████████| 1/1 [00:00<00:00, 14074.85it/s]

Ended : 2023-02-24 21:28:32.550389


In [ ]:
#@title Test BigQuery Permissions etc
# %%script false
sql = """
    SELECT name
    FROM `bigquery-public-data.usa_names.usa_1910_current`
    WHERE state = 'TX'
    LIMIT 100
"""

# Run a Standard SQL query using the environment's default project
# df = pd.read_gbq(sql, dialect='standard')

project_id = 'nipams'
df = pd.read_gbq(sql, project_id=project_id, dialect='standard')
df

,name
0,Mary
1,Ruby
2,Annie
3,Willie
4,Ruth
...,...
95,Leona
96,Lucile
97,Lucy
98,Manuela


# ROUGH

In [ ]:
STOP_HERE

In [ ]:
GBQ_RESULTS_TABLE

'predictions.results_test20_feb24'

In [ ]:
dfResults.to_gbq(GBQ_RESULTS_TABLE,'nipams',
    if_exists='append', table_schema=None,
    # if_exists='append', table_schema=bigquery_schema,
)

100%|██████████| 1/1 [00:00<00:00, 7489.83it/s]


In [ ]:
pipe = pipe_vect_simple
splits = sampleRandTestInds

rfResults1 = experiments.testPipeline(dfImu, dfBp, pipe, splits, targetCol='dbp', verbose=True)
dfBpInd = dfBp.groupby(INDICIES).last()
dfResults1 = create_results_df('dbp', dfBpInd, splits, rfResults1)

rfResults2 = experiments.testPipeline(dfImu, dfBp, pipe, splits, targetCol='sbp', verbose=True)
dfResults2 = create_results_df('sbp', dfBpInd, splits, rfResults2)

dfResults1\
    .merge(dfResults2, left_index=True, right_index=True, suffixes=('','_drop1'))\

In [ ]:
dfResults1\
    .merge(dfResults2, left_index=True, right_index=True, suffixes=('','_drop1'))\

In [ ]:
## create_results_df
target = 'dbp'
results = rfResults1
i=0

col_actual = f'{target}_actuals'
col_pred = f'{target}_preds'
dfTestRes = dfBpInd.copy().assign(fold_number=None, **{col_actual:None, col_pred:None})

df_bp = dfBpInd.loc[splits[i]['test'].set_index(INDICIES).index].reset_index()
dfPreds = pd.DataFrame([results[i]['y_test'].values,results[i]['preds']], index=[col_actual,f'{target}_preds']).T
dfPreds[INDICIES] = splits[i]['test'].reset_index()[INDICIES]


dfRes = pd.merge(df_bp.set_index(INDICIES), dfPreds.set_index(INDICIES), left_index=True, right_index=True,)#.drop_duplicates(INDICIES).set_index(INDICIES)

dfRes = dfRes.assign(fold_number=i)

dfRes

dfTestRes.update(dfRes.drop(columns=BP_COLS))

In [ ]:
display(splits[0]['test'])
display(dfPreds)
display(df_bp)
display("-------------------")

display(splits[0]['test'].drop_duplicates(INDICIES))
display(dfPreds.drop_duplicates(INDICIES))
display(df_bp.drop_duplicates(INDICIES))

# dfTestRes.index.is_unique
# dfTestRes.index.is_unique
# dfRes.index.is_unique




In [ ]:
# dfBp.set_index(INDICIES).loc[sampleRandTestInds[0]['test'].index]
df_bp = dfBp.set_index(INDICIES).loc[sampleRandTestInds[0]['test'].set_index(INDICIES).index].reset_index()
df_bp.groupby(INDICIES).last()

In [ ]:
rfResults1[0]['y_test']

In [ ]:
dfResults1